In [33]:
import os
from dotenv import load_dotenv
load_dotenv()

GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")
GROQ_API_KEY=os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT=os.getenv("LANGCHAIN_PROJECT")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["GROQ_API_KEY"]= GROQ_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]=LANGCHAIN_PROJECT

In [34]:
from langchain_groq import ChatGroq
llm=ChatGroq(model_name="llama3-groq-70b-8192-tool-use-preview")

In [35]:
llm.invoke("hi").content

'Hello! How can I assist you today?'

In [36]:
message=[{"role":"system","content":"you are a helpful assistant"},
         {"role":"user","content":"hi how are you?"}]

result = llm.invoke(message)
print(result.content)

I'm doing great, thank you! How can I assist you today?


In [37]:
import re
import httpx

In [193]:
from typing import Any

class ChatBot:
    def __init__(self,system=""):
        self.system=system
        self.message=[]
        if self.system:
            self.message.append({"role":"system","content":system})
    def __call__(self,message):
        self.message.append({"role":"user","content":message})
        result=self.execute()
        self.message.append({"role":"assistant","content":result})
        return result
        
    def execute(self):
        llm = ChatGroq(model_name="Gemma2-9b-It")
        result = llm.invoke(self.message)
        return result.content
    

In [39]:
assistant=ChatBot(system="you are a helpful assistant")

In [40]:
print(assistant("hi how are you?"))

Hi hi! I'm doing well, thank you. 😊 

How are you today?  




In [41]:
assistant("can you tell me how i can achieve my fitness goal?")

"I'd love to help you achieve your fitness goals!  \n\nTo give you the best advice, I need a little more information.  Tell me:\n\n* **What is your specific fitness goal?** \n\n   * Do you want to lose weight, gain muscle, improve your cardiovascular health, train for a specific event, or something else?\n\n* **What is your current fitness level?**\n\n   * Are you a beginner, intermediate, or advanced? What kind of exercise do you currently do?\n\n* **How much time can you commit to working out each week?**\n\n* **Do you have any injuries or health conditions I should be aware of?**\n\n* **What are your dietary habits like?**\n\nOnce I have a better understanding of your situation, I can provide you with personalized tips and resources to help you reach your goals. 💪  \n\nLet's get started! 🎉\n"

In [166]:
assistant.message

[{'role': 'system', 'content': 'you are a helpful assistant'},
 {'role': 'user', 'content': 'hi how are you?'},
 {'role': 'assistant',
  'content': "Hi hi! I'm doing well, thank you. 😊 \n\nHow are you today?  \n\n"},
 {'role': 'user',
  'content': 'can you tell me how i can achieve my fitness goal?'},
 {'role': 'assistant',
  'content': "I'd love to help you achieve your fitness goals!  \n\nTo give you the best advice, I need a little more information.  Tell me:\n\n* **What is your specific fitness goal?** \n\n   * Do you want to lose weight, gain muscle, improve your cardiovascular health, train for a specific event, or something else?\n\n* **What is your current fitness level?**\n\n   * Are you a beginner, intermediate, or advanced? What kind of exercise do you currently do?\n\n* **How much time can you commit to working out each week?**\n\n* **Do you have any injuries or health conditions I should be aware of?**\n\n* **What are your dietary habits like?**\n\nOnce I have a better und

In [194]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop your output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.


Your available actions are:
calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point
syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

simon_blog_search:
e.g. simon_blog_search: Python
Search Simon's blog for that term

Example session:
Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:
Observation: France is a country. The capital is Paris.

You then output:
Answer: The capital of France is Paris

Please Note: if you get basic conversation questions like "hi","hello","how are you?",\n
you have to answer "hi","hello","i am good".
""".strip()

In [34]:
prompt = """You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop, you output an Answer related to fitness, health, or exercise.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 1800 * 0.2
Runs a calculation (e.g., calorie intake, BMI, workout progress) and returns the result.

fitness_wiki:
e.g. fitness_wiki: Benefits of Cardio
Returns a summary of fitness-related information from a fitness knowledge base (Wikipedia or similar).

exercise_plan:
e.g. exercise_plan: Beginner full-body workout
Returns a recommended workout plan based on the type of exercise or user level.

nutrition_info:
e.g. nutrition_info: Protein sources for vegetarians
Looks up nutritional information or suggestions for diets.

Example session:
Question: What are the benefits of cardio exercises?
Thought: I should look up information about the benefits of cardio exercises.
Action: fitness_wiki: Benefits of Cardio
PAUSE

You will be called again with this:

Observation: Cardio exercises improve cardiovascular health, increase lung capacity, and aid in weight loss.

You then output:

Answer: Cardio exercises improve cardiovascular health, increase lung capacity, and help with weight loss.
""".strip()

In [170]:
action_re = re.compile('^Action: (\w+): (.*)')

In [44]:
import re

# Compiling the regular expression pattern
action_re = re.compile('^Action: (\w+): (.*)')

# Sample strings
text1 = "Action: Move: North"
text2 = "Action: Jump: High"
text3 = "Error: Not an Action"

In [45]:
# Testing the pattern
match1 = action_re.match(text1)

In [47]:
match1

<re.Match object; span=(0, 19), match='Action: Move: North'>

In [53]:
match1.group(1)

'Move'

In [54]:
match1.group(2)

'North'

In [48]:
match2 = action_re.match(text2)

In [49]:
match2

<re.Match object; span=(0, 18), match='Action: Jump: High'>

In [50]:
match3 = action_re.match(text3)

In [58]:
match3

In [59]:
# Extracting the matched groups
if match1:
    print(f"Matched: {match1.group(1)}, {match1.group(2)}")  # Move, North

Matched: Move, North


In [60]:
if match2:
    print(f"Matched: {match2.group(1)}, {match2.group(2)}")  # Jump, High

Matched: Jump, High


In [62]:
if match3:
    print("No match")  # This won't match, as it doesn't start with "Action:"

In [195]:
def wikipedia(q):
    response = httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": q,
        "format": "json"
    })
    return response.json()["query"]["search"][0]["snippet"]

In [196]:
def simon_blog_search(q):
    response = httpx.get("https://datasette.simonwillison.net/simonwillisonblog.json", params={
        "sql": """
        select
          blog_entry.title || ': ' || substr(html_strip_tags(blog_entry.body), 0, 1000) as text,
          blog_entry.created
        from
          blog_entry join blog_entry_fts on blog_entry.rowid = blog_entry_fts.rowid
        where
          blog_entry_fts match escape_fts(:q)
        order by
          blog_entry_fts.rank
        limit
          1
        """.strip(),
        "_shape": "array",
        "q": q,
    })
    return response.json()[0]["text"]

In [197]:
def calculate(numbers):
    return eval(numbers)

In [174]:
calculate("10*7")

70

In [198]:
known_actions = {
    "wikipedia": wikipedia,
    "calculate": calculate,
    "simon_blog_search": simon_blog_search
}

In [181]:
question="hi how are you?"

In [182]:
bot = ChatBot(prompt)

In [183]:
bot

In [189]:
question="hello again?"

In [190]:
next_prompt = question

In [191]:
result = bot(next_prompt)
print(result)

hello!


In [129]:
actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]

In [130]:
actions

[]

In [199]:
def query(question, max_turns=5):
    i = 0
    bot = ChatBot(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: {action_input}")
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = f"Observation: {observation}"
        else:
            return result

In [200]:
query("hi")

Okay, I understand the loop and my available actions. I'm ready for the first question!  



"Okay, I understand the loop and my available actions. I'm ready for the first question!  \n"

In [201]:
query("hello")

Understood. I'm ready for my first question.  



"Understood. I'm ready for my first question.  \n"

In [202]:
query("what you doing?")

Understood.  

Thought: I am an AI assistant waiting for a question.

Action:  None

PAUSE 



'Understood.  \n\nThought: I am an AI assistant waiting for a question.\n\nAction:  None\n\nPAUSE \n'

In [204]:
query("Has Simon written about AI?")

Thought: I need to check Simon's blog for any posts about AI. 
Action: simon_blog_search: AI
PAUSE 

 -- running simon_blog_search AI
Observation: It's OK to call it Artificial Intelligence: Update 9th January 2024: This post was clumsily written and failed to make the point I wanted it to make. I've published a follow-up, What I should have said about the term Artificial Intelligence which you should read instead.
My original post follows.


We need to be having high quality conversations about AI: what it can and can't do, its many risks and pitfalls and how to integrate it into society in the most beneficial ways possible.
Any time I write anything that mentions AI it's inevitable that someone will object to the very usage of the term.
Strawman: "Don't call it AI! It's not actually intelligent - it's just spicy autocomplete."
That strawman is right: it's not "intelligent" in the same way that humans are. And "spicy autocomplete" is actually a pretty good analogy for how a lot of the

'Answer: Yes, Simon has written about AI.  He discusses the term "Artificial Intelligence" and its appropriateness in a blog post titled "It\'s OK to call it Artificial Intelligence". \n'

In [32]:
print(query("Fifteen * twenty five"))

Thought: I need to calculate 15 * 25. 

Action: calculate: 15 * 25 
PAUSE 

 -- running calculate 15 * 25 
Observation: 375


